In [2]:
import csv
import math

import matplotlib.pyplot as plt
import numpy as np

from keras import applications
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras.initializers import glorot_uniform
from keras.applications.vgg16 import preprocess_input
from keras.models import load_model


import cv2
from matplotlib import pyplot as plt
from skimage import color
from skimage.transform import resize

from IPython.display import Audio
from IPython.display import display
import time

Using TensorFlow backend.


In [3]:
import numpy as np

print(len(np.load('pac_arrays/train_all.npy')))
print(len(np.load('pac_arrays/test_all.npy')))
print(len(np.load('pac_arrays/dev_all.npy')))

print(np.load('sub_dirs.npy'))

25740
8580
8580
['02' '04' '06' '08' '11' '15' '23' '39' '52' '59' '62' '63' '72']


In [4]:
import csv
with open("pac_guide.csv") as file:
    reader = csv.reader(file)
    i = 0
    image_paths = []
    camera_nums = []
    sanitation_labels = []
    labels = []
    
    for row in reader:
        index = row[0]
        camera_num = row[1]
        label = row[2]
        filename = row[3]
        image_paths.append(filename)
        labels.append(label)
        #camera_nums.append(camera_num)
        sanitation_labels.append(label)
        #print(index, camera_num, label, filename)
print(len(image_paths)) # number of rows: 112036
#print(len(camera_nums))
print(sanitation_labels[0:5])

sub_dir_hash = {}
for i, sub_dir in enumerate(np.load('sub_dirs.npy')):
    sub_dir_hash[sub_dir] = i

print(sub_dir_hash)

112036
['0', '0', '0', '0', '0']
{'23': 6, '62': 10, '52': 8, '72': 12, '39': 7, '59': 9, '63': 11, '06': 2, '02': 0, '04': 1, '15': 5, '11': 4, '08': 3}


In [5]:
def depth_map_to_image(depth_map):
    img = cv2.normalize(depth_map, depth_map, 0, 1, cv2.NORM_MINMAX)
    img = np.array(img * 255, dtype=np.uint8)
    orig_img = img
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    img = cv2.applyColorMap(img, cv2.COLORMAP_OCEAN)
    #img = resize(img, 224, 224)

    return orig_img, img

def image_generator(indices, batch_size):

    num_batches = int(len(indices) / batch_size)
    
    while True:
        for batch_i in range(num_batches):
            if batch_i == num_batches - 1:
                # special case: return as many as possible
                start_i = batch_i * batch_size
                batch_indices = indices[start_i:]
                
                X = np.zeros((len(batch_indices), 224, 224, 3))
                Y = np.zeros((len(batch_indices), 2)) # Change to one-hot
            
            else:
                start_i = batch_i * batch_size
                end_i = start_i + batch_size

                batch_indices = indices[start_i:end_i]

                X = np.zeros((batch_size, 224, 224, 3))
                Y = np.zeros((batch_size, 2)) # Change to one-hot
            
            for i, index in enumerate(batch_indices):
                #img = image.load_img(image_paths[index], target_size=(224, 224))
                data = np.load(image_paths[int(index)])
                depth_map = data['x'].astype(np.float32)
                orig_img, ocean = depth_map_to_image(depth_map)
                ocean = resize(ocean, (224, 224), mode='constant')
                
                X[i, :, :, :] = ocean
                # Convert to 1 hot vector
                one_hot = np.zeros(2)
                sanitation_label = sanitation_labels[int(index)]
                if sanitation_label == '0':
                    one_hot[0] = 1
                elif sanitation_label == '1':
                    one_hot[1] = 1
                Y[i,:] = one_hot
            
            # use vgg16 preprocessing
            X = preprocess_input(X)
            
            yield (X, Y)

In [21]:
def camera_model(camera_num):
    print(camera_num)
    input_tensor = Input(shape=(224,224,3))
    model = applications.VGG16(weights='imagenet', include_top=False, input_tensor = input_tensor)
    
    # build a classifier model to put on top of the convolutional model
    x = model.output
    x = Flatten(input_shape=(model.output_shape[1:]))(x)
    #x = Dropout(0.2)(x)
    x = Dropout(0.05)(x)
    x = Dense(256, activation='relu', kernel_initializer='glorot_uniform')(x)
    x = Dense(2, activation='softmax', name='output', kernel_initializer='glorot_uniform')(x)

    # add new classifier model on top of convolutional base
    new_model = Model(model.input, x)
    
    for layer in new_model.layers[:19]:
        layer.trainable = False
        
    #new_model = load_model('pca_camera_' + camera_num + '.hdf5')
    
    # RMSprop
    #new_model.compile(loss='categorical_crossentropy',
    #              optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])
    new_model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])

    minibatch_size = 64#64

    train_indices = np.load('pac_arrays/train_' + camera_num + '.npy')
    test_indices = np.load('pac_arrays/test_' + camera_num + '.npy')

    image_generator(train_indices, minibatch_size)

    epochs = 30
    minibatch_size = 64#64

    train_steps = math.ceil(len(train_indices) / minibatch_size)
    test_steps = math.ceil(len(test_indices) / minibatch_size)
    
    num0 = 0
    num1 = 0
    for i in train_indices:
        if labels[i] == '0':
            num0 += 1
        elif labels[i] == '1':
            num1 += 1
    ratio = int(num0 / num1)
    print(ratio)
        
        
        
    
    #class_weight = {0 : 1.,
    #1: ratio}
    class_weight = {0 : 1.,
    1: 250}

    # fine-tune the model
    history = new_model.fit_generator(
        image_generator(train_indices, minibatch_size),
        steps_per_epoch=train_steps,
        epochs=epochs,
        class_weight = class_weight,
        validation_data=image_generator(test_indices, minibatch_size),
        nb_val_samples=test_steps)

    new_model.save('pca_camera_' + camera_num + '.hdf5')

In [9]:
#camera_model('02')
camera_nums = np.load('sub_dirs.npy')
camera_model(camera_nums[0])
#for camera_num in camera_nums:
#    camera_model(camera_num)
sound_file = 'http://www.pacdv.com/sounds/interface_sound_effects/sound82.wav'
display(Audio(url=sound_file, autoplay=True))

02
12
Epoch 1/30


/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:64: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:64: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=11, class_weight={0: 1.0, 1..., steps_per_epoch=31, validation_data=<generator..., epochs=30)`


31/31 [==============================] - 65s - loss: 2.4221 - acc: 0.5352 - val_loss: 2.8442 - val_acc: 0.0884
Epoch 2/30
31/31 [==============================] - 42s - loss: 1.8918 - acc: 0.5229 - val_loss: 0.3133 - val_acc: 0.9130
Epoch 3/30
31/31 [==============================] - 42s - loss: 1.6983 - acc: 0.5682 - val_loss: 2.8588 - val_acc: 0.0870
Epoch 4/30
31/31 [==============================] - 43s - loss: 1.6303 - acc: 0.5763 - val_loss: 0.4667 - val_acc: 0.9126
Epoch 5/30
31/31 [==============================] - 42s - loss: 1.5558 - acc: 0.5836 - val_loss: 0.9677 - val_acc: 0.0870
Epoch 6/30
31/31 [==============================] - 43s - loss: 1.5104 - acc: 0.5847 - val_loss: 0.2483 - val_acc: 0.9144
Epoch 7/30
31/31 [==============================] - 43s - loss: 1.4786 - acc: 0.6132 - val_loss: 0.3473 - val_acc: 0.9116
Epoch 8/30
31/31 [==============================] - 42s - loss: 1.3354 - acc: 0.6295 - val_loss: 0.6462 - val_acc: 0.7293
Epoch 9/30
31/31 [=================

In [10]:
#camera_model('02')
camera_nums = np.load('sub_dirs.npy')
camera_model(camera_nums[1])
#for camera_num in camera_nums:
#    camera_model(camera_num)
sound_file = 'http://www.pacdv.com/sounds/interface_sound_effects/sound82.wav'
display(Audio(url=sound_file, autoplay=True))

04
9
Epoch 1/30


/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:64: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:64: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=11, class_weight={0: 1.0, 1..., steps_per_epoch=31, validation_data=<generator..., epochs=30)`


31/31 [==============================] - 45s - loss: 2.0348 - acc: 0.5362 - val_loss: 0.4193 - val_acc: 0.9088
Epoch 2/30
31/31 [==============================] - 43s - loss: 1.7285 - acc: 0.5333 - val_loss: 0.3226 - val_acc: 0.9088
Epoch 3/30
31/31 [==============================] - 43s - loss: 1.5339 - acc: 0.5789 - val_loss: 0.7164 - val_acc: 0.1285
Epoch 4/30
31/31 [==============================] - 42s - loss: 1.4326 - acc: 0.5638 - val_loss: 0.2839 - val_acc: 0.9102
Epoch 5/30
31/31 [==============================] - 43s - loss: 1.4264 - acc: 0.5905 - val_loss: 0.7731 - val_acc: 0.0884
Epoch 6/30
31/31 [==============================] - 42s - loss: 1.3414 - acc: 0.5998 - val_loss: 0.6584 - val_acc: 0.7514
Epoch 7/30
31/31 [==============================] - 43s - loss: 1.2679 - acc: 0.6335 - val_loss: 0.3840 - val_acc: 0.9337
Epoch 8/30
31/31 [==============================] - 43s - loss: 1.2447 - acc: 0.6501 - val_loss: 0.5482 - val_acc: 0.9422
Epoch 9/30
31/31 [=================

In [11]:
#camera_model('02')
camera_nums = np.load('sub_dirs.npy')
camera_model(camera_nums[2])
#for camera_num in camera_nums:
#    camera_model(camera_num)
sound_file = 'http://www.pacdv.com/sounds/interface_sound_effects/sound82.wav'
display(Audio(url=sound_file, autoplay=True))

06
1
Epoch 1/30


/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:64: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:64: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=11, class_weight={0: 1.0, 1..., steps_per_epoch=31, validation_data=<generator..., epochs=30)`


31/31 [==============================] - 45s - loss: 1.0284 - acc: 0.5427 - val_loss: 0.7431 - val_acc: 0.6174
Epoch 2/30
31/31 [==============================] - 43s - loss: 0.8100 - acc: 0.5533 - val_loss: 0.7831 - val_acc: 0.3895
Epoch 3/30
31/31 [==============================] - 43s - loss: 0.7894 - acc: 0.5618 - val_loss: 0.7897 - val_acc: 0.6116
Epoch 4/30
31/31 [==============================] - 43s - loss: 0.7913 - acc: 0.5555 - val_loss: 0.6682 - val_acc: 0.5193
Epoch 5/30
31/31 [==============================] - 43s - loss: 0.7486 - acc: 0.5784 - val_loss: 0.6573 - val_acc: 0.6116
Epoch 6/30
31/31 [==============================] - 42s - loss: 0.7319 - acc: 0.5754 - val_loss: 0.6108 - val_acc: 0.7320
Epoch 7/30
31/31 [==============================] - 42s - loss: 0.7327 - acc: 0.5946 - val_loss: 0.7384 - val_acc: 0.6146
Epoch 8/30
31/31 [==============================] - 42s - loss: 0.7048 - acc: 0.6046 - val_loss: 0.6018 - val_acc: 0.7887
Epoch 9/30
31/31 [=================

In [12]:
#camera_model('02')
camera_nums = np.load('sub_dirs.npy')
camera_model(camera_nums[3])
#for camera_num in camera_nums:
#    camera_model(camera_num)
sound_file = 'http://www.pacdv.com/sounds/interface_sound_effects/sound82.wav'
display(Audio(url=sound_file, autoplay=True))

08
8
Epoch 1/30


/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:64: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:64: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=11, class_weight={0: 1.0, 1..., steps_per_epoch=31, validation_data=<generator..., epochs=30)`


31/31 [==============================] - 45s - loss: 2.0149 - acc: 0.5351 - val_loss: 1.0330 - val_acc: 0.0981
Epoch 2/30
31/31 [==============================] - 43s - loss: 1.3884 - acc: 0.5917 - val_loss: 0.4278 - val_acc: 0.9213
Epoch 3/30
31/31 [==============================] - 42s - loss: 1.2971 - acc: 0.6531 - val_loss: 0.3744 - val_acc: 0.9268
Epoch 4/30
31/31 [==============================] - 42s - loss: 1.1677 - acc: 0.6914 - val_loss: 0.4381 - val_acc: 0.9337
Epoch 5/30
31/31 [==============================] - 42s - loss: 1.1400 - acc: 0.6861 - val_loss: 0.3117 - val_acc: 0.9378
Epoch 6/30
31/31 [==============================] - 43s - loss: 1.0460 - acc: 0.7241 - val_loss: 0.4141 - val_acc: 0.9116
Epoch 7/30
31/31 [==============================] - 42s - loss: 1.0834 - acc: 0.7319 - val_loss: 0.2007 - val_acc: 0.9213
Epoch 8/30
31/31 [==============================] - 43s - loss: 0.9343 - acc: 0.7548 - val_loss: 0.3878 - val_acc: 0.9116
Epoch 9/30
31/31 [=================

In [13]:
#camera_model('02')
camera_nums = np.load('sub_dirs.npy')
camera_model(camera_nums[4])
#for camera_num in camera_nums:
#    camera_model(camera_num)
sound_file = 'http://www.pacdv.com/sounds/interface_sound_effects/sound82.wav'
display(Audio(url=sound_file, autoplay=True))

11
19
Epoch 1/30


/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:64: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:64: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=11, class_weight={0: 1.0, 1..., steps_per_epoch=31, validation_data=<generator..., epochs=30)`


31/31 [==============================] - 45s - loss: 2.5912 - acc: 0.5451 - val_loss: 1.0387 - val_acc: 0.0401
Epoch 2/30
31/31 [==============================] - 43s - loss: 1.9647 - acc: 0.5471 - val_loss: 0.7385 - val_acc: 0.2721
Epoch 3/30
31/31 [==============================] - 43s - loss: 1.7842 - acc: 0.5736 - val_loss: 0.1624 - val_acc: 0.9597
Epoch 4/30
31/31 [==============================] - 43s - loss: 1.8207 - acc: 0.6257 - val_loss: 1.4775 - val_acc: 0.0345
Epoch 5/30
31/31 [==============================] - 43s - loss: 1.6993 - acc: 0.5862 - val_loss: 0.1966 - val_acc: 0.9583
Epoch 6/30
31/31 [==============================] - 43s - loss: 1.6044 - acc: 0.6602 - val_loss: 2.2410 - val_acc: 0.0345
Epoch 7/30
31/31 [==============================] - 44s - loss: 1.5768 - acc: 0.6413 - val_loss: 0.3686 - val_acc: 0.9220
Epoch 8/30
31/31 [==============================] - 43s - loss: 1.3859 - acc: 0.6992 - val_loss: 0.7019 - val_acc: 0.6105
Epoch 9/30
31/31 [=================

In [22]:
#camera_model('02')
camera_nums = np.load('sub_dirs.npy')
camera_model(camera_nums[5])
#for camera_num in camera_nums:
#    camera_model(camera_num)
sound_file = 'http://www.pacdv.com/sounds/interface_sound_effects/sound82.wav'
display(Audio(url=sound_file, autoplay=True))

15
494
Epoch 1/30


/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:66: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:66: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_data=<generator..., steps_per_epoch=31, validation_steps=11, epochs=30, class_weight={0: 1.0, 1...)`


31/31 [==============================] - 43s - loss: 3.6586 - acc: 0.7645 - val_loss: 0.1369 - val_acc: 0.9931
Epoch 2/30
31/31 [==============================] - 40s - loss: 2.6284 - acc: 0.6416 - val_loss: 0.1200 - val_acc: 0.9945
Epoch 3/30
31/31 [==============================] - 41s - loss: 2.7049 - acc: 0.6803 - val_loss: 1.9194 - val_acc: 0.0041
Epoch 4/30
31/31 [==============================] - 41s - loss: 2.5895 - acc: 0.6668 - val_loss: 1.2722 - val_acc: 0.0055
Epoch 5/30
31/31 [==============================] - 41s - loss: 2.6777 - acc: 0.6236 - val_loss: 1.0605 - val_acc: 0.0166
Epoch 6/30
31/31 [==============================] - 41s - loss: 2.3163 - acc: 0.6348 - val_loss: 0.4360 - val_acc: 0.9959
Epoch 7/30
31/31 [==============================] - 41s - loss: 2.7267 - acc: 0.6420 - val_loss: 0.2249 - val_acc: 0.9959
Epoch 8/30
31/31 [==============================] - 41s - loss: 2.8853 - acc: 0.6484 - val_loss: 0.1319 - val_acc: 0.9931
Epoch 9/30
31/31 [=================

In [15]:
#camera_model('02')
camera_nums = np.load('sub_dirs.npy')
camera_model(camera_nums[6])
#for camera_num in camera_nums:
#    camera_model(camera_num)
sound_file = 'http://www.pacdv.com/sounds/interface_sound_effects/sound82.wav'
display(Audio(url=sound_file, autoplay=True))

23
3
Epoch 1/30


/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:64: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:64: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=11, class_weight={0: 1.0, 1..., steps_per_epoch=31, validation_data=<generator..., epochs=30)`


31/31 [==============================] - 45s - loss: 1.7795 - acc: 0.5201 - val_loss: 1.4953 - val_acc: 0.2251
Epoch 2/30
31/31 [==============================] - 43s - loss: 1.2657 - acc: 0.5327 - val_loss: 0.4521 - val_acc: 0.7693
Epoch 3/30
31/31 [==============================] - 43s - loss: 1.1587 - acc: 0.5768 - val_loss: 0.9264 - val_acc: 0.2251
Epoch 4/30
31/31 [==============================] - 43s - loss: 1.0286 - acc: 0.6103 - val_loss: 0.4176 - val_acc: 0.7997
Epoch 5/30
31/31 [==============================] - 43s - loss: 0.9671 - acc: 0.6515 - val_loss: 0.9340 - val_acc: 0.4558
Epoch 6/30
31/31 [==============================] - 43s - loss: 0.9023 - acc: 0.6615 - val_loss: 0.3717 - val_acc: 0.7804
Epoch 7/30
31/31 [==============================] - 42s - loss: 0.8767 - acc: 0.6902 - val_loss: 0.7554 - val_acc: 0.7196
Epoch 8/30
31/31 [==============================] - 43s - loss: 0.8369 - acc: 0.7092 - val_loss: 0.3545 - val_acc: 0.7873
Epoch 9/30
31/31 [=================

In [16]:
#camera_model('02')
camera_nums = np.load('sub_dirs.npy')
camera_model(camera_nums[7])
#for camera_num in camera_nums:
#    camera_model(camera_num)
sound_file = 'http://www.pacdv.com/sounds/interface_sound_effects/sound82.wav'
display(Audio(url=sound_file, autoplay=True))

39
151
Epoch 1/30


/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:64: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:64: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=11, class_weight={0: 1.0, 1..., steps_per_epoch=31, validation_data=<generator..., epochs=30)`


31/31 [==============================] - 45s - loss: 3.6487 - acc: 0.5537 - val_loss: 0.3132 - val_acc: 0.9917
Epoch 2/30
31/31 [==============================] - 43s - loss: 2.5739 - acc: 0.5764 - val_loss: 3.0696 - val_acc: 0.0097
Epoch 3/30
31/31 [==============================] - 43s - loss: 2.5260 - acc: 0.5839 - val_loss: 1.7603 - val_acc: 0.0081
Epoch 4/30
31/31 [==============================] - 43s - loss: 2.3125 - acc: 0.5918 - val_loss: 0.4752 - val_acc: 0.9903
Epoch 5/30
31/31 [==============================] - 43s - loss: 2.4455 - acc: 0.5779 - val_loss: 0.1968 - val_acc: 0.9919
Epoch 6/30
31/31 [==============================] - 43s - loss: 2.3086 - acc: 0.6048 - val_loss: 0.1085 - val_acc: 0.9903
Epoch 7/30
31/31 [==============================] - 43s - loss: 2.1754 - acc: 0.6328 - val_loss: 0.0870 - val_acc: 0.9903
Epoch 8/30
31/31 [==============================] - 43s - loss: 2.7933 - acc: 0.6348 - val_loss: 4.1997 - val_acc: 0.0097
Epoch 9/30
31/31 [=================

In [17]:
#camera_model('02')
camera_nums = np.load('sub_dirs.npy')
camera_model(camera_nums[8])
#for camera_num in camera_nums:
#    camera_model(camera_num)
sound_file = 'http://www.pacdv.com/sounds/interface_sound_effects/sound82.wav'
display(Audio(url=sound_file, autoplay=True))

52
5
Epoch 1/30


/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:64: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:64: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=11, class_weight={0: 1.0, 1..., steps_per_epoch=31, validation_data=<generator..., epochs=30)`


31/31 [==============================] - 45s - loss: 1.8639 - acc: 0.5612 - val_loss: 1.9856 - val_acc: 0.1395
Epoch 2/30
31/31 [==============================] - 43s - loss: 1.4106 - acc: 0.5646 - val_loss: 0.3803 - val_acc: 0.8633
Epoch 3/30
31/31 [==============================] - 44s - loss: 1.3407 - acc: 0.5889 - val_loss: 0.4605 - val_acc: 0.8589
Epoch 4/30
31/31 [==============================] - 43s - loss: 1.3630 - acc: 0.6014 - val_loss: 0.4167 - val_acc: 0.8633
Epoch 5/30
31/31 [==============================] - 43s - loss: 1.3437 - acc: 0.6281 - val_loss: 3.1174 - val_acc: 0.1411
Epoch 6/30
31/31 [==============================] - 43s - loss: 1.3281 - acc: 0.5709 - val_loss: 0.3944 - val_acc: 0.8605
Epoch 7/30
31/31 [==============================] - 43s - loss: 1.2424 - acc: 0.6221 - val_loss: 1.4143 - val_acc: 0.1411
Epoch 8/30
31/31 [==============================] - 43s - loss: 1.2409 - acc: 0.6100 - val_loss: 0.4464 - val_acc: 0.8564
Epoch 9/30
31/31 [=================

In [18]:
#camera_model('02')
camera_nums = np.load('sub_dirs.npy')
camera_model(camera_nums[9])
#for camera_num in camera_nums:
#    camera_model(camera_num)
sound_file = 'http://www.pacdv.com/sounds/interface_sound_effects/sound82.wav'
display(Audio(url=sound_file, autoplay=True))

59
72
Epoch 1/30


/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:64: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:64: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=11, class_weight={0: 1.0, 1..., steps_per_epoch=31, validation_data=<generator..., epochs=30)`


31/31 [==============================] - 45s - loss: 2.8082 - acc: 0.5680 - val_loss: 0.1901 - val_acc: 0.9834
Epoch 2/30
31/31 [==============================] - 42s - loss: 2.3604 - acc: 0.5026 - val_loss: 1.6015 - val_acc: 0.0152
Epoch 3/30
31/31 [==============================] - 43s - loss: 2.1211 - acc: 0.4965 - val_loss: 0.2487 - val_acc: 0.9848
Epoch 4/30
31/31 [==============================] - 43s - loss: 2.0441 - acc: 0.5472 - val_loss: 0.1357 - val_acc: 0.9862
Epoch 5/30
31/31 [==============================] - 43s - loss: 2.0425 - acc: 0.5613 - val_loss: 0.1004 - val_acc: 0.9848
Epoch 6/30
31/31 [==============================] - 43s - loss: 1.9689 - acc: 0.5435 - val_loss: 1.7726 - val_acc: 0.0180
Epoch 7/30
31/31 [==============================] - 43s - loss: 2.1238 - acc: 0.5363 - val_loss: 0.1614 - val_acc: 0.9848
Epoch 8/30
31/31 [==============================] - 43s - loss: 1.7098 - acc: 0.5791 - val_loss: 0.1014 - val_acc: 0.9820
Epoch 9/30
31/31 [=================

In [19]:
#camera_model('02')
camera_nums = np.load('sub_dirs.npy')
camera_model(camera_nums[10])
#for camera_num in camera_nums:
#    camera_model(camera_num)
sound_file = 'http://www.pacdv.com/sounds/interface_sound_effects/sound82.wav'
display(Audio(url=sound_file, autoplay=True))

62
3
Epoch 1/30


/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:64: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:64: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=11, class_weight={0: 1.0, 1..., steps_per_epoch=31, validation_data=<generator..., epochs=30)`


31/31 [==============================] - 45s - loss: 1.7040 - acc: 0.5372 - val_loss: 0.8422 - val_acc: 0.2486
Epoch 2/30
31/31 [==============================] - 43s - loss: 1.2467 - acc: 0.5501 - val_loss: 0.5108 - val_acc: 0.7555
Epoch 3/30
31/31 [==============================] - 44s - loss: 1.1353 - acc: 0.5799 - val_loss: 0.5707 - val_acc: 0.7513
Epoch 4/30
31/31 [==============================] - 43s - loss: 1.0905 - acc: 0.6008 - val_loss: 0.6125 - val_acc: 0.8122
Epoch 5/30
31/31 [==============================] - 43s - loss: 1.0649 - acc: 0.6236 - val_loss: 0.4652 - val_acc: 0.7804
Epoch 6/30
31/31 [==============================] - 43s - loss: 1.0154 - acc: 0.6229 - val_loss: 0.5076 - val_acc: 0.8467
Epoch 7/30
31/31 [==============================] - 43s - loss: 0.9928 - acc: 0.6475 - val_loss: 0.6192 - val_acc: 0.7141
Epoch 8/30
31/31 [==============================] - 43s - loss: 0.9648 - acc: 0.6568 - val_loss: 0.4482 - val_acc: 0.7831
Epoch 9/30
31/31 [=================

In [20]:
#camera_model('02')
camera_nums = np.load('sub_dirs.npy')
camera_model(camera_nums[11])
#for camera_num in camera_nums:
#    camera_model(camera_num)
sound_file = 'http://www.pacdv.com/sounds/interface_sound_effects/sound82.wav'
display(Audio(url=sound_file, autoplay=True))

63
2
Epoch 1/30


/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:64: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:64: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=11, class_weight={0: 1.0, 1..., steps_per_epoch=31, validation_data=<generator..., epochs=30)`


31/31 [==============================] - 45s - loss: 1.4941 - acc: 0.5099 - val_loss: 0.6723 - val_acc: 0.7610
Epoch 2/30
31/31 [==============================] - 42s - loss: 1.1921 - acc: 0.5241 - val_loss: 1.2083 - val_acc: 0.3218
Epoch 3/30
31/31 [==============================] - 43s - loss: 1.1224 - acc: 0.5186 - val_loss: 0.8222 - val_acc: 0.6851
Epoch 4/30
31/31 [==============================] - 42s - loss: 1.1128 - acc: 0.5512 - val_loss: 1.1807 - val_acc: 0.3066
Epoch 5/30
31/31 [==============================] - 43s - loss: 1.0750 - acc: 0.5265 - val_loss: 0.7186 - val_acc: 0.6809
Epoch 6/30
31/31 [==============================] - 43s - loss: 1.0648 - acc: 0.5493 - val_loss: 0.5834 - val_acc: 0.7155
Epoch 7/30
31/31 [==============================] - 43s - loss: 1.0288 - acc: 0.5400 - val_loss: 0.6149 - val_acc: 0.7887
Epoch 8/30
31/31 [==============================] - 43s - loss: 1.0033 - acc: 0.5588 - val_loss: 0.7741 - val_acc: 0.3149
Epoch 9/30
31/31 [=================

In [21]:
#camera_model('02')
camera_nums = np.load('sub_dirs.npy')
camera_model(camera_nums[12])
#for camera_num in camera_nums:
#    camera_model(camera_num)
sound_file = 'http://www.pacdv.com/sounds/interface_sound_effects/sound82.wav'
display(Audio(url=sound_file, autoplay=True))

72
2
Epoch 1/30


/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:64: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:64: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=11, class_weight={0: 1.0, 1..., steps_per_epoch=31, validation_data=<generator..., epochs=30)`


31/31 [==============================] - 45s - loss: 1.5817 - acc: 0.5258 - val_loss: 0.5868 - val_acc: 0.7279
Epoch 2/30
31/31 [==============================] - 43s - loss: 1.1974 - acc: 0.5384 - val_loss: 0.7576 - val_acc: 0.2859
Epoch 3/30
31/31 [==============================] - 43s - loss: 1.1488 - acc: 0.5354 - val_loss: 0.5619 - val_acc: 0.7312
Epoch 4/30
31/31 [==============================] - 43s - loss: 1.0918 - acc: 0.5477 - val_loss: 0.9061 - val_acc: 0.2804
Epoch 5/30
31/31 [==============================] - 44s - loss: 1.0707 - acc: 0.5717 - val_loss: 0.5687 - val_acc: 0.7379
Epoch 6/30
31/31 [==============================] - 43s - loss: 1.0531 - acc: 0.5817 - val_loss: 0.8535 - val_acc: 0.2859
Epoch 7/30
31/31 [==============================] - 43s - loss: 1.0105 - acc: 0.5599 - val_loss: 0.6625 - val_acc: 0.7312
Epoch 8/30
31/31 [==============================] - 43s - loss: 1.0292 - acc: 0.5869 - val_loss: 1.2430 - val_acc: 0.2804
Epoch 9/30
31/31 [=================

In [28]:
def test_image_generator(indices, batch_size):

    num_batches = int(len(indices) / batch_size)
    
    while True:
        for batch_i in range(num_batches):
            if batch_i == num_batches - 1:
                # special case: return as many as possible
                start_i = batch_i * batch_size
                batch_indices = indices[start_i:]
                
                X = np.zeros((len(batch_indices), 224, 224, 3))
                Y = np.zeros((len(batch_indices), 2)) # Change to one-hot
            
            else:
                start_i = batch_i * batch_size
                end_i = start_i + batch_size

                batch_indices = indices[start_i:end_i]

                X = np.zeros((batch_size, 224, 224, 3))
                Y = np.zeros((batch_size, 2)) # Change to one-hot
            
            for i, index in enumerate(batch_indices):
                #img = image.load_img(image_paths[index], target_size=(224, 224))
                data = np.load(image_paths[int(index)])
                depth_map = data['x'].astype(np.float32)
                orig_img, ocean = depth_map_to_image(depth_map)
                ocean = resize(ocean, (224, 224), mode='constant')
                
                X[i, :, :, :] = ocean
                # Convert to 1 hot vector
                one_hot = np.zeros(2)
                sanitation_label = sanitation_labels[int(index)]
                zero_counter = 0
                one_counter = 0
                if sanitation_label == '0':
                    one_hot[0] = 1
                    zero_counter+=1
                elif sanitation_label == '1':
                    one_hot[1] = 1
                    one_counter += 1
                Y[i,:] = one_hot
            
            # use vgg16 preprocessing
            X = preprocess_input(X)
            
            ratio = int(zero_counter / one_counter)

            #class_weight = {0 : 1.,
            #1: ratio}
            class_weight = {0 : 1.,
            1: ratio}
            
            yield (X, Y)

In [30]:
camera_num = '02'
new_model = load_model('pca_camera_' + camera_num + '.hdf5')
    
# RMSprop
#new_model.compile(loss='categorical_crossentropy',
#              optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])
new_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])

minibatch_size = 64#64

train_indices = np.load('pac_arrays/train_' + camera_num + '.npy')
test_indices = np.load('pac_arrays/test_' + camera_num + '.npy')

#image_generator(train_indices, minibatch_size)

epochs = 1
minibatch_size = 64#64

train_steps = math.ceil(len(train_indices) / minibatch_size)
test_steps = math.ceil(len(test_indices) / minibatch_size)

num0 = 0
num1 = 0
for i in train_indices:
    if labels[i] == '0':
        num0 += 1
    elif labels[i] == '1':
        num1 += 1
train_ratio = int(num0 / num1)

num0 = 0
num1 = 0
for i in test_indices:
    if labels[i] == '0':
        num0 += 1
    elif labels[i] == '1':
        num1 += 1
test_ratio = int(num0 / num1)




#class_weight = {0 : 1.,
#1: ratio}
train_weight = {0 : 1.,
1: train_ratio}

test_weight = {0 : 1.,
1: test_ratio}

# fine-tune the model
history = new_model.fit_generator(
    image_generator(train_indices, minibatch_size),
    steps_per_epoch=train_steps,
    epochs=epochs,
    class_weight = train_weight
    )

history = new_model.fit_generator(
    image_generator(test_indices, minibatch_size),
    steps_per_epoch=test_steps,
    epochs=epochs,
    class_weight = test_weight
    )



Epoch 1/1
31/31 [==============================] - 30s - loss: 1.1638 - acc: 0.7455    
Epoch 1/1
11/11 [==============================] - 11s - loss: 0.9007 - acc: 0.7710    


In [31]:
camera_num = '04'
new_model = load_model('pca_camera_' + camera_num + '.hdf5')
    
# RMSprop
#new_model.compile(loss='categorical_crossentropy',
#              optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])
new_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])

minibatch_size = 64#64

train_indices = np.load('pac_arrays/train_' + camera_num + '.npy')
test_indices = np.load('pac_arrays/test_' + camera_num + '.npy')

#image_generator(train_indices, minibatch_size)

epochs = 1
minibatch_size = 64#64

train_steps = math.ceil(len(train_indices) / minibatch_size)
test_steps = math.ceil(len(test_indices) / minibatch_size)

num0 = 0
num1 = 0
for i in train_indices:
    if labels[i] == '0':
        num0 += 1
    elif labels[i] == '1':
        num1 += 1
train_ratio = int(num0 / num1)

num0 = 0
num1 = 0
for i in test_indices:
    if labels[i] == '0':
        num0 += 1
    elif labels[i] == '1':
        num1 += 1
test_ratio = int(num0 / num1)




#class_weight = {0 : 1.,
#1: ratio}
train_weight = {0 : 1.,
1: train_ratio}

test_weight = {0 : 1.,
1: test_ratio}

# fine-tune the model
history = new_model.fit_generator(
    image_generator(train_indices, minibatch_size),
    steps_per_epoch=train_steps,
    epochs=epochs,
    class_weight = train_weight
    )

history = new_model.fit_generator(
    image_generator(test_indices, minibatch_size),
    steps_per_epoch=test_steps,
    epochs=epochs,
    class_weight = test_weight
    )



Epoch 1/1
31/31 [==============================] - 30s - loss: 1.1137 - acc: 0.7196    
Epoch 1/1
11/11 [==============================] - 11s - loss: 0.7966 - acc: 0.7678    


In [32]:
camera_num = '06'
new_model = load_model('pca_camera_' + camera_num + '.hdf5')
    
# RMSprop
#new_model.compile(loss='categorical_crossentropy',
#              optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])
new_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])

minibatch_size = 64#64

train_indices = np.load('pac_arrays/train_' + camera_num + '.npy')
test_indices = np.load('pac_arrays/test_' + camera_num + '.npy')

#image_generator(train_indices, minibatch_size)

epochs = 1
minibatch_size = 64#64

train_steps = math.ceil(len(train_indices) / minibatch_size)
test_steps = math.ceil(len(test_indices) / minibatch_size)

num0 = 0
num1 = 0
for i in train_indices:
    if labels[i] == '0':
        num0 += 1
    elif labels[i] == '1':
        num1 += 1
train_ratio = int(num0 / num1)

num0 = 0
num1 = 0
for i in test_indices:
    if labels[i] == '0':
        num0 += 1
    elif labels[i] == '1':
        num1 += 1
test_ratio = int(num0 / num1)




#class_weight = {0 : 1.,
#1: ratio}
train_weight = {0 : 1.,
1: train_ratio}

test_weight = {0 : 1.,
1: test_ratio}

# fine-tune the model
history = new_model.fit_generator(
    image_generator(train_indices, minibatch_size),
    steps_per_epoch=train_steps,
    epochs=epochs,
    class_weight = train_weight
    )

history = new_model.fit_generator(
    image_generator(test_indices, minibatch_size),
    steps_per_epoch=test_steps,
    epochs=epochs,
    class_weight = test_weight
    )



Epoch 1/1
31/31 [==============================] - 30s - loss: 0.6433 - acc: 0.6931    
Epoch 1/1
11/11 [==============================] - 11s - loss: 0.5559 - acc: 0.6812    


In [33]:
camera_num = '08'
new_model = load_model('pca_camera_' + camera_num + '.hdf5')
    
# RMSprop
#new_model.compile(loss='categorical_crossentropy',
#              optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])
new_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])

minibatch_size = 64#64

train_indices = np.load('pac_arrays/train_' + camera_num + '.npy')
test_indices = np.load('pac_arrays/test_' + camera_num + '.npy')

#image_generator(train_indices, minibatch_size)

epochs = 1
minibatch_size = 64#64

train_steps = math.ceil(len(train_indices) / minibatch_size)
test_steps = math.ceil(len(test_indices) / minibatch_size)

num0 = 0
num1 = 0
for i in train_indices:
    if labels[i] == '0':
        num0 += 1
    elif labels[i] == '1':
        num1 += 1
train_ratio = int(num0 / num1)

num0 = 0
num1 = 0
for i in test_indices:
    if labels[i] == '0':
        num0 += 1
    elif labels[i] == '1':
        num1 += 1
test_ratio = int(num0 / num1)




#class_weight = {0 : 1.,
#1: ratio}
train_weight = {0 : 1.,
1: train_ratio}

test_weight = {0 : 1.,
1: test_ratio}

# fine-tune the model
history = new_model.fit_generator(
    image_generator(train_indices, minibatch_size),
    steps_per_epoch=train_steps,
    epochs=epochs,
    class_weight = train_weight
    )

history = new_model.fit_generator(
    image_generator(test_indices, minibatch_size),
    steps_per_epoch=test_steps,
    epochs=epochs,
    class_weight = test_weight
    )



Epoch 1/1
31/31 [==============================] - 31s - loss: 0.8768 - acc: 0.8229    
Epoch 1/1
11/11 [==============================] - 11s - loss: 0.5580 - acc: 0.8721    


In [34]:
camera_num = '11'
new_model = load_model('pca_camera_' + camera_num + '.hdf5')
    
# RMSprop
#new_model.compile(loss='categorical_crossentropy',
#              optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])
new_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])

minibatch_size = 64#64

train_indices = np.load('pac_arrays/train_' + camera_num + '.npy')
test_indices = np.load('pac_arrays/test_' + camera_num + '.npy')

#image_generator(train_indices, minibatch_size)

epochs = 1
minibatch_size = 64#64

train_steps = math.ceil(len(train_indices) / minibatch_size)
test_steps = math.ceil(len(test_indices) / minibatch_size)

num0 = 0
num1 = 0
for i in train_indices:
    if labels[i] == '0':
        num0 += 1
    elif labels[i] == '1':
        num1 += 1
train_ratio = int(num0 / num1)

num0 = 0
num1 = 0
for i in test_indices:
    if labels[i] == '0':
        num0 += 1
    elif labels[i] == '1':
        num1 += 1
test_ratio = int(num0 / num1)




#class_weight = {0 : 1.,
#1: ratio}
train_weight = {0 : 1.,
1: train_ratio}

test_weight = {0 : 1.,
1: test_ratio}

# fine-tune the model
history = new_model.fit_generator(
    image_generator(train_indices, minibatch_size),
    steps_per_epoch=train_steps,
    epochs=epochs,
    class_weight = train_weight
    )

history = new_model.fit_generator(
    image_generator(test_indices, minibatch_size),
    steps_per_epoch=test_steps,
    epochs=epochs,
    class_weight = test_weight
    )



Epoch 1/1
31/31 [==============================] - 31s - loss: 1.1246 - acc: 0.7587    
Epoch 1/1
11/11 [==============================] - 11s - loss: 1.1986 - acc: 0.7403    


In [36]:
camera_num = '15'
new_model = load_model('pca_camera_' + camera_num + '.hdf5')
    
# RMSprop
#new_model.compile(loss='categorical_crossentropy',
#              optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])
new_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])

minibatch_size = 64#64

train_indices = np.load('pac_arrays/train_' + camera_num + '.npy')
test_indices = np.load('pac_arrays/test_' + camera_num + '.npy')

#image_generator(train_indices, minibatch_size)

epochs = 1
minibatch_size = 64#64

train_steps = math.ceil(len(train_indices) / minibatch_size)
test_steps = math.ceil(len(test_indices) / minibatch_size)

num0 = 0
num1 = 0
for i in train_indices:
    if labels[i] == '0':
        num0 += 1
    elif labels[i] == '1':
        num1 += 1
train_ratio = int(num0 / num1)

num0 = 0
num1 = 0
for i in test_indices:
    if labels[i] == '0':
        num0 += 1
    elif labels[i] == '1':
        num1 += 1
test_ratio = int(num0 / num1)




#class_weight = {0 : 1.,
#1: ratio}
train_weight = {0 : 1.,
1: train_ratio}

test_weight = {0 : 1.,
1: test_ratio}

# fine-tune the model
history = new_model.fit_generator(
    image_generator(train_indices, minibatch_size),
    steps_per_epoch=train_steps,
    epochs=epochs,
    class_weight = train_weight
    )

history = new_model.fit_generator(
    image_generator(test_indices, minibatch_size),
    steps_per_epoch=test_steps,
    epochs=epochs,
    class_weight = test_weight
    )



Epoch 1/1
31/31 [==============================] - 31s - loss: 6.0743 - acc: 0.6674    
Epoch 1/1
11/11 [==============================] - 11s - loss: 2.5017 - acc: 0.6315    


In [37]:
camera_num = '23'
new_model = load_model('pca_camera_' + camera_num + '.hdf5')
    
# RMSprop
#new_model.compile(loss='categorical_crossentropy',
#              optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])
new_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])

minibatch_size = 64#64

train_indices = np.load('pac_arrays/train_' + camera_num + '.npy')
test_indices = np.load('pac_arrays/test_' + camera_num + '.npy')

#image_generator(train_indices, minibatch_size)

epochs = 1
minibatch_size = 64#64

train_steps = math.ceil(len(train_indices) / minibatch_size)
test_steps = math.ceil(len(test_indices) / minibatch_size)

num0 = 0
num1 = 0
for i in train_indices:
    if labels[i] == '0':
        num0 += 1
    elif labels[i] == '1':
        num1 += 1
train_ratio = int(num0 / num1)

num0 = 0
num1 = 0
for i in test_indices:
    if labels[i] == '0':
        num0 += 1
    elif labels[i] == '1':
        num1 += 1
test_ratio = int(num0 / num1)




#class_weight = {0 : 1.,
#1: ratio}
train_weight = {0 : 1.,
1: train_ratio}

test_weight = {0 : 1.,
1: test_ratio}

# fine-tune the model
history = new_model.fit_generator(
    image_generator(train_indices, minibatch_size),
    steps_per_epoch=train_steps,
    epochs=epochs,
    class_weight = train_weight
    )

history = new_model.fit_generator(
    image_generator(test_indices, minibatch_size),
    steps_per_epoch=test_steps,
    epochs=epochs,
    class_weight = test_weight
    )



Epoch 1/1
31/31 [==============================] - 31s - loss: 0.6770 - acc: 0.7732    
Epoch 1/1
11/11 [==============================] - 11s - loss: 0.6240 - acc: 0.7809    


In [38]:
camera_num = '39'
new_model = load_model('pca_camera_' + camera_num + '.hdf5')
    
# RMSprop
#new_model.compile(loss='categorical_crossentropy',
#              optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])
new_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])

minibatch_size = 64#64

train_indices = np.load('pac_arrays/train_' + camera_num + '.npy')
test_indices = np.load('pac_arrays/test_' + camera_num + '.npy')

#image_generator(train_indices, minibatch_size)

epochs = 1
minibatch_size = 64#64

train_steps = math.ceil(len(train_indices) / minibatch_size)
test_steps = math.ceil(len(test_indices) / minibatch_size)

num0 = 0
num1 = 0
for i in train_indices:
    if labels[i] == '0':
        num0 += 1
    elif labels[i] == '1':
        num1 += 1
train_ratio = int(num0 / num1)

num0 = 0
num1 = 0
for i in test_indices:
    if labels[i] == '0':
        num0 += 1
    elif labels[i] == '1':
        num1 += 1
test_ratio = int(num0 / num1)




#class_weight = {0 : 1.,
#1: ratio}
train_weight = {0 : 1.,
1: train_ratio}

test_weight = {0 : 1.,
1: test_ratio}

# fine-tune the model
history = new_model.fit_generator(
    image_generator(train_indices, minibatch_size),
    steps_per_epoch=train_steps,
    epochs=epochs,
    class_weight = train_weight
    )

history = new_model.fit_generator(
    image_generator(test_indices, minibatch_size),
    steps_per_epoch=test_steps,
    epochs=epochs,
    class_weight = test_weight
    )



Epoch 1/1
31/31 [==============================] - 31s - loss: 3.1614 - acc: 0.6425    
Epoch 1/1
11/11 [==============================] - 11s - loss: 1.0874 - acc: 0.7394    


In [41]:
camera_num = '52'
new_model = load_model('pca_camera_' + camera_num + '.hdf5')
    
# RMSprop
#new_model.compile(loss='categorical_crossentropy',
#              optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])
new_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])

minibatch_size = 64#64

train_indices = np.load('pac_arrays/train_' + camera_num + '.npy')
test_indices = np.load('pac_arrays/test_' + camera_num + '.npy')

#image_generator(train_indices, minibatch_size)

epochs = 1
minibatch_size = 64#64

train_steps = math.ceil(len(train_indices) / minibatch_size)
test_steps = math.ceil(len(test_indices) / minibatch_size)

num0 = 0
num1 = 0
for i in train_indices:
    if labels[i] == '0':
        num0 += 1
    elif labels[i] == '1':
        num1 += 1
train_ratio = int(num0 / num1)

num0 = 0
num1 = 0
for i in test_indices:
    if labels[i] == '0':
        num0 += 1
    elif labels[i] == '1':
        num1 += 1
test_ratio = int(num0 / num1)




#class_weight = {0 : 1.,
#1: ratio}
train_weight = {0 : 1.,
1: train_ratio}

test_weight = {0 : 1.,
1: test_ratio}

print(train_ratio, test_ratio)

# fine-tune the model
history = new_model.fit_generator(
    image_generator(train_indices, minibatch_size),
    steps_per_epoch=train_steps,
    epochs=epochs,
    class_weight = train_weight
    )

history = new_model.fit_generator(
    image_generator(test_indices, minibatch_size),
    steps_per_epoch=test_steps,
    epochs=epochs,
    class_weight = test_weight
    )



5 6
Epoch 1/1
31/31 [==============================] - 31s - loss: 1.0828 - acc: 0.6977    
Epoch 1/1
11/11 [==============================] - 11s - loss: 1.0944 - acc: 0.6536    


In [42]:
camera_num = '59'
new_model = load_model('pca_camera_' + camera_num + '.hdf5')
    
# RMSprop
#new_model.compile(loss='categorical_crossentropy',
#              optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])
new_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])

minibatch_size = 64#64

train_indices = np.load('pac_arrays/train_' + camera_num + '.npy')
test_indices = np.load('pac_arrays/test_' + camera_num + '.npy')

#image_generator(train_indices, minibatch_size)

epochs = 1
minibatch_size = 64#64

train_steps = math.ceil(len(train_indices) / minibatch_size)
test_steps = math.ceil(len(test_indices) / minibatch_size)

num0 = 0
num1 = 0
for i in train_indices:
    if labels[i] == '0':
        num0 += 1
    elif labels[i] == '1':
        num1 += 1
train_ratio = int(num0 / num1)

num0 = 0
num1 = 0
for i in test_indices:
    if labels[i] == '0':
        num0 += 1
    elif labels[i] == '1':
        num1 += 1
test_ratio = int(num0 / num1)

print(train_ratio, test_ratio)




#class_weight = {0 : 1.,
#1: ratio}
train_weight = {0 : 1.,
1: train_ratio}

test_weight = {0 : 1.,
1: test_ratio}

# fine-tune the model
history = new_model.fit_generator(
    image_generator(train_indices, minibatch_size),
    steps_per_epoch=train_steps,
    epochs=epochs,
    class_weight = train_weight
    )

history = new_model.fit_generator(
    image_generator(test_indices, minibatch_size),
    steps_per_epoch=test_steps,
    epochs=epochs,
    class_weight = test_weight
    )



72 65
Epoch 1/1
31/31 [==============================] - 31s - loss: 2.0904 - acc: 0.6793    
Epoch 1/1
11/11 [==============================] - 11s - loss: 1.1743 - acc: 0.7027    


In [43]:
camera_num = '62'
new_model = load_model('pca_camera_' + camera_num + '.hdf5')
    
# RMSprop
#new_model.compile(loss='categorical_crossentropy',
#              optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])
new_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])

minibatch_size = 64#64

train_indices = np.load('pac_arrays/train_' + camera_num + '.npy')
test_indices = np.load('pac_arrays/test_' + camera_num + '.npy')

#image_generator(train_indices, minibatch_size)

epochs = 1
minibatch_size = 64#64

train_steps = math.ceil(len(train_indices) / minibatch_size)
test_steps = math.ceil(len(test_indices) / minibatch_size)

num0 = 0
num1 = 0
for i in train_indices:
    if labels[i] == '0':
        num0 += 1
    elif labels[i] == '1':
        num1 += 1
train_ratio = int(num0 / num1)

num0 = 0
num1 = 0
for i in test_indices:
    if labels[i] == '0':
        num0 += 1
    elif labels[i] == '1':
        num1 += 1
test_ratio = int(num0 / num1)


print(train_ratio, test_ratio)


#class_weight = {0 : 1.,
#1: ratio}
train_weight = {0 : 1.,
1: train_ratio}

test_weight = {0 : 1.,
1: test_ratio}

# fine-tune the model
history = new_model.fit_generator(
    image_generator(train_indices, minibatch_size),
    steps_per_epoch=train_steps,
    epochs=epochs,
    class_weight = train_weight
    )

history = new_model.fit_generator(
    image_generator(test_indices, minibatch_size),
    steps_per_epoch=test_steps,
    epochs=epochs,
    class_weight = test_weight
    )



3 3
Epoch 1/1
31/31 [==============================] - 31s - loss: 0.8433 - acc: 0.7461    
Epoch 1/1
11/11 [==============================] - 11s - loss: 0.7543 - acc: 0.7516    


In [6]:
camera_num = '63'
new_model = load_model('pca_camera_' + camera_num + '.hdf5')
    
# RMSprop
#new_model.compile(loss='categorical_crossentropy',
#              optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])
new_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])

minibatch_size = 64#64

train_indices = np.load('pac_arrays/train_' + camera_num + '.npy')
test_indices = np.load('pac_arrays/test_' + camera_num + '.npy')

#image_generator(train_indices, minibatch_size)

epochs = 1
minibatch_size = 64#64

train_steps = math.ceil(len(train_indices) / minibatch_size)
test_steps = math.ceil(len(test_indices) / minibatch_size)

num0 = 0
num1 = 0
for i in train_indices:
    if labels[i] == '0':
        num0 += 1
    elif labels[i] == '1':
        num1 += 1
train_ratio = int(num0 / num1)

num0 = 0
num1 = 0
for i in test_indices:
    if labels[i] == '0':
        num0 += 1
    elif labels[i] == '1':
        num1 += 1
test_ratio = int(num0 / num1)


print(train_ratio, test_ratio)


#class_weight = {0 : 1.,
#1: ratio}
train_weight = {0 : 1.,
1: train_ratio}

test_weight = {0 : 1.,
1: test_ratio}

# fine-tune the model
history = new_model.fit_generator(
    image_generator(train_indices, minibatch_size),
    steps_per_epoch=train_steps,
    epochs=epochs,
    class_weight = train_weight
    )

history = new_model.fit_generator(
    image_generator(test_indices, minibatch_size),
    steps_per_epoch=test_steps,
    epochs=epochs,
    class_weight = test_weight
    )



2 2
Epoch 1/1
31/31 [==============================] - 44s - loss: 0.9023 - acc: 0.6594    
Epoch 1/1
11/11 [==============================] - 17s - loss: 0.7945 - acc: 0.6480    


In [7]:
camera_num = '72'
new_model = load_model('pca_camera_' + camera_num + '.hdf5')
    
# RMSprop
#new_model.compile(loss='categorical_crossentropy',
#              optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])
new_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])

minibatch_size = 64#64

train_indices = np.load('pac_arrays/train_' + camera_num + '.npy')
test_indices = np.load('pac_arrays/test_' + camera_num + '.npy')

#image_generator(train_indices, minibatch_size)

epochs = 1
minibatch_size = 64#64

train_steps = math.ceil(len(train_indices) / minibatch_size)
test_steps = math.ceil(len(test_indices) / minibatch_size)

num0 = 0
num1 = 0
for i in train_indices:
    if labels[i] == '0':
        num0 += 1
    elif labels[i] == '1':
        num1 += 1
train_ratio = int(num0 / num1)

num0 = 0
num1 = 0
for i in test_indices:
    if labels[i] == '0':
        num0 += 1
    elif labels[i] == '1':
        num1 += 1
test_ratio = int(num0 / num1)


print(train_ratio, test_ratio)


#class_weight = {0 : 1.,
#1: ratio}
train_weight = {0 : 1.,
1: train_ratio}

test_weight = {0 : 1.,
1: test_ratio}

# fine-tune the model
history = new_model.fit_generator(
    image_generator(train_indices, minibatch_size),
    steps_per_epoch=train_steps,
    epochs=epochs,
    class_weight = train_weight
    )

history = new_model.fit_generator(
    image_generator(test_indices, minibatch_size),
    steps_per_epoch=test_steps,
    epochs=epochs,
    class_weight = test_weight
    )



2 2
Epoch 1/1
31/31 [==============================] - 30s - loss: 0.9471 - acc: 0.6253    
Epoch 1/1
11/11 [==============================] - 11s - loss: 0.9245 - acc: 0.6163    
